In [8]:
# Library import

import json
from pprint import pprint
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [300]:
with open(r'result.json', 'r', encoding='utf-8') as f:
    result = json.load(f)
with open(r'full_data.json', 'r', encoding='utf-8') as f:
    full_data = json.load(f)

In [226]:
# import json
# from pprint import pprint

# with open(r'recent.json', 'r', encoding='utf-8') as f:
#     json_0 = json.load(f)

# pprint(json_0)

In [310]:
def re_col(js):
    date = js['date'][:7]   # Insert date information for each song (olny using year-month)
    
    for item in js['data']:
        item['date'] = date
        for key in ['image', 'last_week_rank','peak_rank']:   # Delete some features
            item.pop(key, None)
            
    js = js['data']   # Change the dictionary structure to the list>dictionary structure
    return js

In [311]:
# JSON files downloaded through the Billboard API (file divided by week)
# Import json file name in folder
import os
folder = r"\HOT100_json"
f_names = os.listdir(folder)

# print(f_names)

In [312]:
# Gathering data from each divided file and integrating it into one
full_data = []

for f_name in f_names:
    path = os.path.join(folder, f_name)
    with open(path, 'r', encoding='utf-8') as f:
        f_json = json.load(f)

    add_data = re_col(f_json)
    full_data.extend(add_data)

In [1]:
# print('Total number of data:', len(full_data))
# full_data

In [314]:
# Delete overlapping songs within the same month (because we will look up the data at least on a monthly basis)
# Choose the one with the highest 'weeks_on_chart'
import pandas as pd

def remove_duplicates(data):
    df = pd.DataFrame(data)
    
    df_sort = df.sort_values(by=['date','weeks_on_chart'], ascending=False)
    df_drop = df_sort.drop_duplicates(subset=['date', 'name', 'artist'])
    dic_list = df_drop.to_dict(orient='records')

    ## After deduplication, check the number of songs per month (make sure the amount of data is sufficient)
    # count_per_date = df_drop['date'].value_counts()
    # print(count_per_date)
    
    return dic_list

In [3]:
remove_dup_data = remove_duplicates(full_data)

# print('Total number of data since deduplication:', len(remove_dup_data))
# remove_dup_data

In [316]:
# Lyrics collection function
from googlesearch import search
import random

def get_lyrics(dic_data):
    
    name = dic_data['name']
    artist = dic_data['artist']
    query = f'{artist} {name} lyrics genius'
    
    ## Url should be accurate, so if there is an abnormality in the title or singer name, url works x
    ## Many url is not working due to special marking such as featuring
    # search_url = f"https://genius.com/{query.replace(' ', '-')}"

    # Google Search + Connect to Top Link
    for url in search(query, num_results=1):
        search_url = url
        break
        
    driver.get(search_url)
    time.sleep(random.uniform(2, 5))

    path_0 = '//*[@id="lyrics-root"]/div[1]'
    path_4 = '//*[@id="lyrics-root"]/div[5]'

    lyrics_texts = []
    
    elem_0 = driver.find_element(By.XPATH, path_0)
    elem_4 = driver.find_element(By.XPATH, path_4)
    lyrics_texts = [elem_0.text, elem_4.text]

    return '\n'.join(lyrics_texts)

In [317]:
# It is recommended to reduce the number of crawls as much as possible
# Deduplication in the whole (after the first batch of data will be filled with the rest of the duplicated data)
unique_songs = {}

for song in full_data:
    key = (song['name'], song['artist'])
    if key not in unique_songs:
        unique_songs[key] = song  # Save the first song only

result = list(unique_songs.values())
len(result)

805

In [374]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# user-agent
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) "
                     "Chrome/114.0.0.0 Safari/537.36")

service = Service(r'D:\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)   

for dic in result:
    if dic.get('lyrics'):    #Skip if it has already collected lyrics
        continue
    else:
        try:
            dic['lyrics'] = get_lyrics(dic)
        except Exception as e: 
            print(f"{dic['name']}, {dic['artist']} Cannot find lyrics :", e)
            
driver.quit()

Tu Name, Fuerza Regida Cannot find lyrics : 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DFuerza%2BRegida%2BTu%2BName%2Blyrics%2Bgenius%26num%3D3%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgSNSPsiGJD24cIGIjBQK7hv--QIqcGmzVI6hczdhpw0cXi4Rd4bYJDL9yqFrHOkOeUHCr-tdR0x0ufHSa0yAnJSWgFD


In [366]:
# Result checking
print(result[85])

{'name': 'Back Then Right Now', 'artist': 'Tyler Hubbard', 'rank': 86, 'weeks_on_chart': 11, 'date': '2024-06', 'lyrics': '4 Contributors\nBack Then Right Now Lyrics\n[Intro]\nBack then, right now\n\n[Verse 1]\nBack when that blacktop was just a gravel road\nBack when that Walmart was just a fishin\' hole\nBack when my ol\' guitar had some brand-new strings\nYes, it did\nBack when a work truck was just a stick shift\nBack when you took a pic and didn\'t have to post it\nBack when a Baptist church was where we learned to sing\nYeah, we did\n\n[Chorus]\nI could use some back then, right now\n\'98 Chevy with the tailgate down\nFM only with the gold up loud\nBurnin\' up the night, innocent and wild\nI could use a little more wide-open\nBack when all I wanted was the hand I was holdin\'\nLivin\' in the moment with the good-time crowd\nMakin\' life count\nDamn, I could use a little more\nBack then, right now\nBack then, right now\n[Verse 2]\nBack when a, "How ya been?" came through a phone c

In [368]:
# Save JSON file
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)
with open('full_data.json', 'w', encoding='utf-8') as f:
    json.dump(full_data, f, ensure_ascii=False, indent=4)